In [19]:
import json
import re
from datetime import datetime
import pandas as pd

In [20]:
# JSON 파일 경로 설정
json_file_path = '../law_data/law_preprocessing.json'

# JSON 파일 불러오기
with open(json_file_path, 'r') as file:
    data = json.load(file)


In [21]:
addenda_dict = {key: data[key]['addenda'] for key in data if 'addenda' in data[key]}

In [22]:
# 저장할 파일 경로 설정
output_file_path = '../law_data/addenda_data.json'

# addenda_dict를 JSON 파일로 저장
with open(output_file_path, 'w') as output_file:
    json.dump(addenda_dict, output_file, ensure_ascii=False, indent=4)


In [23]:
# 각 key, key_2, article의 key와 value를 하나의 행으로 저장할 리스트
rows = []

# 데이터 변환
for key in addenda_dict.keys():
    buchick = addenda_dict[key]
    for key_2 in buchick.keys():
        date_match = re.search(r'\d{4}\. \d{1,2}\. \d{1,2}\.', key_2)
        if date_match:
            # 문자열을 datetime 객체로 변환
            date_str = date_match.group(0)
            date_obj = datetime.strptime(date_str, '%Y. %m. %d.')
            # 원하는 형식 (예: YYYY-MM-DD)으로 변환
            formatted_date = date_obj.strftime('%Y-%m-%d')
        article = buchick[key_2]
        if isinstance(article, dict):
            for article_key, article_value in article.items():
                rows.append({
                    'law_name': key.replace(" ", ""),  # 공백 제거
                    'effective_date': formatted_date,
                    'article': article_key,
                    'passage': article_value
                })
        else:
            # article이 딕셔너리가 아닐 경우
            rows.append({
                'law_name': key.replace(" ", ""),  # 공백 제거
                'effective_date': formatted_date,
                'article': None,
                'passage': article
            })

# DataFrame으로 변환
df = pd.DataFrame(rows)

In [24]:
df.to_csv('../law_data/addenda.csv', encoding='utf-8-sig', lineterminator='\n')